In [1]:
import compare_MSAG_range as cmp
import geopandas as gpd
import os
import pandas as pd

In [18]:
gdf = cmp.getAddressPoints()[0]

In [19]:
gdf.fillna("", inplace=True)

In [20]:
gdf.DI.unique()

array(['', 'S', 'N', 'E', 'W', 'SW', 'NW'], dtype=object)

In [23]:
gdf.head()

,ESN,HouseNbr,DI,STREET,STCODE,CITY,GEOM,CTCODE,KEY
0,54,18008,,SIUSLAW,RD,WALTON,POINT (4100415.106 836505.844),WAN,54__SIUSLAW_RD_WAN
1,56,83908,,HWY 101,,FLORENCE,POINT (3969716.775 839107.124),FLO,56__HWY 101__FLO
2,56,83520,,HWY 101,,FLORENCE,POINT (3968343.749 835007.831),FLO,56__HWY 101__FLO
3,56,83394,,HWY 101,,FLORENCE,POINT (3968070.590 833992.111),FLO,56__HWY 101__FLO
4,56,83380,,HWY 101,,FLORENCE,POINT (3968010.308 833853.040),FLO,56__HWY 101__FLO


In [5]:
intrado = cmp.getIntrado()[2]

In [7]:
selectedCols = ['ESN', 'DI', 'STREET', 'STCODE', 'CTCODE']

In [22]:
gdf['KEY'] = gdf[selectedCols].apply(lambda row: str(row.ESN) + '_' + row.DI + '_' + row.STREET + '_' +  row.STCODE + '_' +  row.CTCODE, axis=1)

In [14]:
intrado['KEY'] = intrado[selectedCols].apply(lambda row: str(row.ESN) + '_' + row.DI + '_' + row.STREET + '_' +  row.STCODE + '_' +  row.CTCODE, axis=1)

In [15]:
intrado.head()

,ESN,DI,STREET,STCODE,CTCODE,LOW,HIGH,KEY
0,1,E,10TH,AVE,EUG,55,401,1_E_10TH_AVE_EUG
7018,1,E,11TH,ALY,EUG,337,337,1_E_11TH_ALY_EUG
23,1,E,11TH,AVE,EUG,23,950,1_E_11TH_AVE_EUG
7020,1,E,12TH,ALY,EUG,143,838,1_E_12TH_ALY_EUG
40,1,E,12TH,AVE,EUG,146,876,1_E_12TH_AVE_EUG


In [16]:
len(intrado.KEY.unique())

6008

In [17]:
intrado.shape

(7973, 8)

In [33]:
commonKeys = [key for key in gdf.KEY.unique() if key in intrado.KEY.unique()]
len(commonKeys)

5662

In [25]:
len([key for key in gdf.KEY.unique() if key not in intrado.KEY.unique()])

282

In [28]:
len([key for key in intrado.KEY.unique() if key not in gdf.KEY.unique()])

346

In [31]:
intrado[intrado.duplicated(subset=['KEY'])].KEY.values[0]

'1_E_25TH_AVE_EUG'

In [93]:
intrado[intrado.KEY=='50__FIR_ST_WEL']

,ESN,DI,STREET,STCODE,CTCODE,LOW,HIGH,KEY
2386,50,,FIR,ST,WEL,82855,82878,50__FIR_ST_WEL


In [107]:
key = '50__FIR_ST_WEL'

In [94]:
ap_sdf = gdf[gdf.KEY == key]

In [95]:
hnbr_min = ap_sdf.HouseNbr.values.min()

In [96]:
hnbr_max = ap_sdf.HouseNbr.values.max()

In [102]:
ie_sdf = intrado[intrado.KEY==key]

In [103]:
ie_sdf.shape[0] == 1

True

In [104]:
ie_sdf

,ESN,DI,STREET,STCODE,CTCODE,LOW,HIGH,KEY
2386,50,,FIR,ST,WEL,82855,82878,50__FIR_ST_WEL


In [105]:
hnbr_min >= ie_sdf.LOW.values[0]

False

In [106]:
hnbr_max <= ie_sdf.HIGH.values[0]

True

In [109]:
ie_sdf.HIGH.values[0]

82878

In [110]:
ie_sdf.LOW.values[0]

82855

In [115]:
gdf.loc[((gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr) & (gdf.HouseNbr >= ie_sdf.LOW.values[0])), 'HouseNbr']

320    82855
321    82855
708    82878
712    82872
714    82866
716    82862
797    82855
832    82855
833    82855
834    82855
835    82855
836    82855
837    82855
838    82855
Name: HouseNbr, dtype: int64

In [59]:
gdf.loc[gdf.KEY == key, 'DIFF'] = 1

In [60]:
gdf.loc[gdf.KEY == key, 'MATCH'] = 'Exactly matched'

In [62]:
key = '1_E_25TH_AVE_EUG'

In [69]:
hnbr_min 

50

In [70]:
hnbr_max

3640

In [74]:
ie_sdf.HIGH.values[ie_sdf.shape[0]-1]

3640

In [73]:
ie_sdf.shape[0]

2

In [76]:
ap_sdf.shape

(76, 11)

In [77]:
ap_sdf.head()

,ESN,HouseNbr,DI,STREET,STCODE,CITY,GEOM,CTCODE,KEY,DIFF,MATCH
79782,1,95,E,25TH,AVE,EUGENE,POINT (4240235.064 872412.420),EUG,1_E_25TH_AVE_EUG,NaN,NaN
79786,1,145,E,25TH,AVE,EUGENE,POINT (4240449.069 872402.181),EUG,1_E_25TH_AVE_EUG,NaN,NaN
79788,1,163,E,25TH,AVE,EUGENE,POINT (4240524.951 872399.083),EUG,1_E_25TH_AVE_EUG,NaN,NaN
79789,1,195,E,25TH,AVE,EUGENE,POINT (4240588.538 872392.268),EUG,1_E_25TH_AVE_EUG,NaN,NaN
79795,1,50,E,25TH,AVE,EUGENE,POINT (4240126.873 872295.902),EUG,1_E_25TH_AVE_EUG,NaN,NaN


In [85]:
ie_sdf.HIGH.values[0]

3612

In [82]:
def withinRange(x, ie_sdf):
    for i in range(ie_sdf.shape[0]):
        if ie_sdf.HIGH.values[i] >= x >= ie_sdf.LOW.values[i]:
            diff = 1
            match = 'Matched in one of the multiple ranges'
            return diff, match
    diff = 4
    match = 'Outside of one of the multiple ranges'
    return diff, match

In [80]:
list(range(0, ie_sdf.shape[0]))

[0, 1]

In [88]:
ap_sdf.HouseNbr.apply(lambda x: withinRange(x, ie_sdf=ie_sdf)[0])

79782     1
79786     1
79788     1
79789     1
79795     1
         ..
90761     1
133865    1
134796    1
134800    1
134802    1
Name: HouseNbr, Length: 76, dtype: int64

In [89]:
import timeit

In [92]:
start = timeit.default_timer()
for key in commonKeys:
    print(key)
    ap_sdf = gdf[gdf.KEY == key]
    hnbr_min = ap_sdf.HouseNbr.values.min()
    hnbr_max = ap_sdf.HouseNbr.values.max()
    ie_sdf = intrado[intrado.KEY==key]
    if ie_sdf.shape[0] == 1:
        if hnbr_min >= ie_sdf.LOW.values[0] and hnbr_max <= ie_sdf.HIGH.values[0]:
            gdf.loc[gdf.KEY == key, 'DIFF'] = 1
            gdf.loc[gdf.KEY == key, 'MATCH'] = 'Exactly matched'
        elif hnbr_max > ie_sdf.HIGH.values[0] and hnbr_min >= ie_sdf.LOW.values[0]:
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'DIFF'] = 2
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'MATCH'] = 'Outside of house number range in the high value'
            gdf.loc[(gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr >= ie_sdf.LOW.values[0]), 'DIFF'] = 1
            gdf.loc[(gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr >= ie_sdf.LOW.values[0]), 'MATCH'] = 'Matched, within the house number range, but the high end in the same street is outside of range'
        elif hnbr_min < ie_sdf.LOW.values[0] and hnbr_max <= ie_sdf.HIGH.values[0]:
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'DIFF'] = 3
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'MATCH'] = 'Outside of house number range in the low value'
            gdf.loc[(gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr >= ie_sdf.LOW.values[0]), 'DIFF'] = 1
            gdf.loc[(gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr >= ie_sdf.LOW.values[0]), 'MATCH'] = 'Matched, within the house number range, but the low end in the same street is outside of range'
        elif hnbr_max > ie_sdf.HIGH.values[0] and hnbr_min < ie_sdf.LOW.values[0]:
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'DIFF'] = 2
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[0]), 'MATCH'] = 'Outside of house number range in the high value'
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'DIFF'] = 3
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'MATCH'] = 'Outside of house number range in the low value'
            gdf.loc[(gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr >= ie_sdf.LOW.values[0]), 'DIFF'] = 1
            gdf.loc[(gdf.KEY == key) & (ie_sdf.HIGH.values[0] >= gdf.HouseNbr >= ie_sdf.LOW.values[0]), 'MATCH'] = 'Matched, within the house number range, but both high and low ends in the same street are outside of range'
    else:
        if hnbr_max > ie_sdf.HIGH.values[ie_sdf.shape[0]-1]:
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[ie_sdf.shape[0]-1]), 'DIFF'] = 2
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr > ie_sdf.HIGH.values[ie_sdf.shape[0]-1]), 'MATCH'] = 'Outside of the multiple house number ranges in the high value'
        elif hnbr_min < ie_sdf.LOW.values[0]:
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'DIFF'] = 3
            gdf.loc[(gdf.KEY == key) & (gdf.HouseNbr < ie_sdf.LOW.values[0]), 'MATCH'] = 'Outside of the multiple house number ranges in the low value'
        else:
            gdf.loc[gdf.KEY == key, 'DIFF'] = ap_sdf.HouseNbr.apply(lambda x: withinRange(x, ie_sdf=ie_sdf)[0])
            gdf.loc[gdf.KEY == key, 'MATCH'] = ap_sdf.HouseNbr.apply(lambda x: withinRange(x, ie_sdf=ie_sdf)[1])
stop = timeit.default_timer()
print(stop - start)

54__SIUSLAW_RD_WAN
56__HWY 101__FLO
56_S_CANARY_RD_WEL
56__ERHART_RD_FLO
56__SILTCOOS STATION_RD_WEL
56__FIDDLE CREEK_RD_WEL
56__SILTCOOS BEACH ACCESS_RD_FLO
56__LOWER FIDDLE CREEK_RD_WEL
56__LAKE_ST_FLO
56__HWY 101__WEL
54__JERNIGAN ISLAND__WEL
56__3RD_ST_FLO
56__SEAVIEW_LN_FLO
56__BOY SCOUT_RD_FLO
56__MAPLE_DR_FLO
50__HWY 101__FLO
50__BUCKSKIN BOB__FLO
52__BOOTH ISLAND__WEL
50__SALAL_CT_FLO
50__CLOUD NINE_RD_FLO
50__CLEAR LAKE_RD_FLO
50__RUSSELL_DR_FLO
50__KIECHLE ARM_RD_FLO
50__RIO_RD_FLO
50__ERHART_RD_FLO
50__ALDER_CT_FLO
50__WOODLAND_LN_FLO
50__JENSEN_LN_FLO
50__BERRY_LN_FLO
50__MANZANITA_DR_FLO
50__SOUTH COVE_WAY_FLO
50__BRENTWOOD_DR_FLO
50__SOUTH COVE_DR_FLO
50_N_PIONEER_RD_FLO
50__SPRUCE_LN_FLO
50__HILLTOP_DR_FLO
50__PARKWAY_DR_FLO
50__KENDALL_LN_FLO
50__DOROTHY_LN_FLO
50__DARLINGS_LP_FLO
50_S_PIONEER_RD_FLO
50__PACIFIC_AVE_WEL
50__SPRUCE_ST_WEL
50__LAKE_BLVD_WEL
50__SUMMIT_AVE_WEL
50__MEADOW LAKE_WAY_FLO
50__CLEARWATER COVE_WAY_FLO
50__SAUTER_DR_FLO
50__LAUREL_AVE_WEL
50__FIR_

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().